In [ ]:
mport os, glob
import sys
import copy 
import pydicom
import scipy
import scipy.misc
import numpy as np
import cv2
import imageio
from scipy.ndimage import rotate

In [ ]:
# CHANGE DATASET STRUCTURE

for folder in os.listdir("Dataset"):
#     print(folder)
    x = folder.split('db')
    folder_path = os.path.join('Dataset', folder)
#     for subfolder in os.listdir(folder_path):
#     subfolder_path = os.path.join(folder_path,subfolder)
    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path,subfolder)
        if os.path.isdir(subfolder_path):
            for sub_sub in os.listdir(subfolder_path):
                if os.path.isdir(os.path.join(subfolder_path, sub_sub)):
                    for image in os.listdir(os.path.join(subfolder_path, sub_sub)):
                        src= os.path.join(subfolder_path, sub_sub, image)
                        dst = os.path.join(subfolder_path, sub_sub, x[1]+'_'+image)
                        os.rename(src, dst)
                        
                src= os.path.join(subfolder_path, sub_sub)
                dst = os.path.join(subfolder_path, x[1]+'_'+sub_sub)
                os.rename(src, dst)

In [ ]:
# COUNT THE NUMBER OF SCANS FOR 1 PATIENT TO APPEND ON THEM
def count_scans_startwith(directory, prefix):
    count = 0
    for file in os.listdir(directory):
        if file.startswith(prefix+'_'):
            count+=1
    return count

In [ ]:
def merge_livertumors(scans_path, masks_path):
    for scan in os.listdir(scans_path):
        patient_id = scan.split('_')[0]
        tumor_volume = None
        for mask_dir in os.listdir(masks_path):
            if mask_dir.startswith(patient_id+'_livertumor'):
                current_tumor = pydicom.dcmread(os.path.join(masks_path, mask_dir, scan)).pixel_array
                current_tumor = current_tumor/255.0
                current_tumor = np.clip(current_tumor,0,1)
                tumor_volume = current_tumor if tumor_volume is None else np.logical_or(tumor_volume,current_tumor)
    
        if tumor_volume is None:
            tumor_volume = np.zeros((512,512))
        tumor_volume = [[j*255 for j in i] for i in tumor_volume]
        tumor_volume = np.array(tumor_volume)
        tumor_volume = tumor_volume.astype(np.uint8)
        imageio.imwrite(os.path.join(masks_path, 'merged_livertumors', scan+'.jpg'), tumor_volume

In [ ]:
# REFLECT IMAGE AND MASK TO AUGMENT DATA
def reflect_dicom(src_img, src_mask, src_liver_mask, train_path, masks_path, liver_mask_path, patient_id, patient_imgs_count):
    reflected_img = copy.deepcopy(src_img)
    reflected_img.PixelData = np.fliplr(reflected_img.pixel_array).tobytes()
    

    reflected_liver_mask = copy.deepcopy(src_liver_mask)
    reflected_liver_mask.PixelData = np.fliplr(reflected_liver_mask.pixel_array).tobytes()
    reflected_liver_mask.save_as(os.path.join(liver_mask_path, patient_id+'_image_'+str(patient_imgs_count)+'_aug'))
            
    reflected_mask = np.fliplr(src_mask)
    
   
    reflected_img.save_as(os.path.join(train_path, patient_id+'_image_'+str(patient_imgs_count)+'_aug'))

    reflected_mask = reflected_mask.astype(np.uint8)
    imageio.imwrite(os.path.join(masks_path, 'merged_livertumors', patient_id+'_image_'+str(patient_imgs_count)+'_aug.jpg'), reflected_mask)
                    
    
# ROTATE IMAGE AND MASK TO AUGMENT DATA
def rotate_dicom(src_img, src_mask, src_liver_mask, train_path, masks_path, liver_mask_path, patient_id, patient_imgs_count, angle):
    rotated_img = copy.deepcopy(src_img)
    rotated_img.PixelData = rotate(rotated_img.pixel_array, angle, reshape=False).tobytes()
    

    rotated_liver_mask = copy.deepcopy(src_liver_mask)
    rotated_liver_mask.PixelData = rotate(rotated_liver_mask.pixel_array, angle, reshape=False).tobytes()
    rotated_liver_mask.save_as(os.path.join(liver_mask_path, patient_id+'_image_'+str(patient_imgs_count)+'_aug'))
    
    rotated_mask = rotate(src_mask, angle, reshape=False)
    

    rotated_img.save_as(os.path.join(train_path, patient_id+'_image_'+str(patient_imgs_count)+'_aug'))
    
    rotated_mask = rotated_mask.astype(np.uint8)
    imageio.imwrite(os.path.join(masks_path, 'merged_livertumors', patient_id+'_image_'+str(patient_imgs_count)+'_aug.jpg'), rotated_mask)    
    

In [ ]:
def augment_dicom(train_path, masks_path):
    rotation_angles = [x * 90 for x in range(4) if x > 0]
    train_files = copy.deepcopy(os.listdir(train_path))
    
    for scan in train_files:
        mask_path = os.path.join(masks_path, 'merged_livertumors', scan+'.jpg')
        tumor_mask = cv2.imread(mask_path)
        if 1 in tumor_mask:
            patient_id = scan.split('_')[0]
            patient_imgs_count = count_scans_startwith(train_path, patient_id)
            original_img = pydicom.dcmread(os.path.join(train_path, scan)) 
            liver_mask_path = os.path.join(masks_path, patient_id+'_liver')
            liver_mask = pydicom.dcmread(os.path.join(liver_mask_path, scan))
            reflect_dicom(original_img, tumor_mask, liver_mask, train_path, masks_path, liver_mask_path, patient_id, patient_imgs_count)
            
            for angle in rotation_angles:
                patient_imgs_count += 1 
                rotate_dicom(original_img, tumor_mask, liver_mask, train_path, masks_path, liver_mask_path, patient_id, patient_imgs_count, angle)
        

In [ ]:
masks_path = os.path.join('train','masks')
training_path = os.path.join('train','patients')

augment_dicom(training_path, masks_path)
# merge_livertumors(training_path, masks_path)
print('done')

In [ ]:
count = 0
for mask in os.listdir(os.path.join(masks_path,'merged_livertumors')):
    tumor_mask = cv2.imread(os.path.join(masks_path,'merged_livertumors', mask))
    if 1 in tumor_mask:
        count += 1
print(count)
# 568

In [ ]:
# REMOVE AUGMENTED DATA
for scan in os.listdir(training_path):
    if scan.endswith('_aug'):
        os.remove(os.path.join(training_path,scan))


for mask in os.listdir(os.path.join(masks_path, 'merged_livertumors')):
    if mask.endswith('_aug.jpg'):
        os.remove(os.path.join(masks_path, 'merged_livertumors', mask))
print('done')